In [2]:
import pandas as pd
import zipfile
import os
from io import TextIOWrapper
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Load Metadata
metadata = pd.read_csv("metadata.csv") 
# Storing whether each participant is Spanish Bilingual or not in a dictionary 
label_lookup = metadata.set_index("participant")["spanish"].astype(int).to_dict()

# Identify participants
# Participants who only know English
#only_english = metadata[metadata[['spanish', 'german', 'french']].sum(axis=1) == 0]['participant'].astype(str).tolist()
# Participants who know English and Spanish
#bilingual_spanish = metadata[metadata['spanish'] == 1]['participant'].astype(str).tolist()

 


In [7]:
# Folder where CSVs are extracted
erp_folder = "EEG_Measurements"

# ERP row range for 300–500ms - key window for analysis (N400)
start_row, end_row = 102, 154

# Store results from each EEG file
feature_rows = []

for file in os.listdir(erp_folder):
    if not file.endswith(".csv"):
        continue
     #only focusing on these 2 file types to target Spanish Bilingualism 
    if "spanish-english_translation" not in file and "spanish-english_unrelated" not in file:
        continue

    participant_id = int(file.split("_")[-1].split(".")[0])
    condition = "translation" if "translation" in file else "unrelated"
    word = file.split("_")[0].lower()

    df = pd.read_csv(os.path.join(erp_folder, file))
    df_window = df.iloc[start_row:end_row] #target window
    avg_vals = df_window.mean(numeric_only=True) #averaging values across electrodes 

    row = {
        "participant": participant_id,
        "word": word,
        "condition": condition,
        "label": label_lookup.get(participant_id)
    }
    #adding averaged EEG features
    row.update(avg_vals.to_dict()) 
    feature_rows.append(row)

eeg_df = pd.DataFrame(feature_rows)
eeg_df.shape


(1596, 34)